In [1]:
import pandas as pd
import numpy as np
import _thread
import os
import urllib3
urllib3.disable_warnings()
from bs4 import BeautifulSoup
import string

URL = "https://www.letras.mus.br"

In [2]:
letras = string.ascii_uppercase + '1'

In [3]:
def save(filename, df):
    if os.path.exists(filename):
        df.to_csv(filename, mode='a', index=False, header=False)
    else:
        df.to_csv(filename, index=False)

In [4]:
def gravar(dados, letra):
    df = pd.DataFrame(np.array(dados), columns=['url', 'artist', 'title_music', 'original_lyrics', 'translated_lyrics'])
    save("musicas-{}.csv".format(letra), df)

In [5]:
def obter_links_e_nomes(letra):
    links = []
    try:
        print('Obtendo letra {}.'.format(letra))
        http = urllib3.PoolManager()
        url = URL+"/letra/"+letra+"/artistas.html"
        html = http.request('GET', url).data
        soup = BeautifulSoup(html, 'html.parser')
        soup = soup.find('ul',{'class': "cnt-list cnt-list--col3"})
        if soup:
            for link in soup.find_all('a'):
                if len(links) > 1:
                    gravar(links, letra)
                    links = []
                url = URL+link.get('href')
                html = http.request('GET', url).data
                soup = BeautifulSoup(html, 'html.parser')
                for a in soup.find_all('div',{'class':'artista-top'}):
                    for link in a.find_all('a'):
                        if 'contribuicoes' in link.get('href'):
                            continue
                        url = URL+link.get('href')
                        musica = link.text
                        artista = soup.find_all('h1')[-1].text
                        html = http.request('GET', url+ "traducao.html").data
                        
                        soup = BeautifulSoup(html, 'html.parser')
                        orig = soup.find('div',{'class': "cnt-trad_l"})
                        trad = soup.find('div',{'class': "cnt-trad_r"})
                        if orig != None and trad != None:
                            links.append((url, artista, musica, orig.text, trad.text))
                        else:
                            orig = soup.find('div',{'class': "cnt-letra"})
                            links.append((url, artista, musica, orig.text, None))
        print('Thread da letra {} finalizada.'.format(letra))
    except Exception:
        import traceback
        print(traceback.format_exc())

In [6]:
try:
    for letra in letras:
        _thread.start_new_thread(obter_links_e_nomes, (letra,) )
except Exception as e:
    print ("Error: unable to start thread: "+ str(e))

Obtendo letra A.
Obtendo letra G.
Obtendo letra F.
Obtendo letra N.
Obtendo letra K.
Obtendo letra T.
Obtendo letra W.
Obtendo letra Z.
Obtendo letra D.
Obtendo letra I.
Obtendo letra B.
Obtendo letra P.
Obtendo letra U.
Obtendo letra Y.
Obtendo letra C.
Obtendo letra J.
Obtendo letra E.
Obtendo letra L.
Obtendo letra 1.
Obtendo letra H.
Obtendo letra V.Obtendo letra Q.
Obtendo letra X.
Obtendo letra S.
Obtendo letra M.

Obtendo letra R.
Obtendo letra O.


In [ ]:
links = []
http = urllib3.PoolManager()
url = URL+"/os-qdelicias/ela-me-envolve/" + "traducao.html"
html = http.request('GET', url).data

soup = BeautifulSoup(html, 'html.parser')
orig = soup.find('div',{'class': "cnt-trad_l"})
trad = soup.find('div',{'class': "cnt-trad_r"})
if orig != None and trad != None:
    print(orig.text)
    print(trad.text)
else:
    orig = soup.find('div',{'class': "cnt-letra"})
    print(orig.text)

In [ ]:
soup.find_all('h1')[-1].text